# Loan Repayment 
## Exploratory Data Analysis (EDA)

In [14]:
import os
# Change directory to this file's directory
this_path = globals()['_dh'][-1]
os.chdir(this_path)
print("This file's directory:", os.getcwd())
# Change current path to parent of this file's directory
# to access all modules from parent
os.chdir('..')
source_path = os.getcwd()
print("Parent directory:", source_path)

This file's directory: /home/mattkhoo/Git-Loan-Repayment-EDA-Predictive/notebooks
Parent directory: /home/mattkhoo/Git-Loan-Repayment-EDA-Predictive


In [2]:
from main.data_loader import DataLoader
import main.viz_utils as viz
%matplotlib inline

## Load Loan Data

In [21]:
# Load feather data
os.chdir(source_path)
loan_feather_path = os.path.join(str(os.getcwd()),'data-feathers', 'loan') 
print(loan_feather_path)
# Create data loader for loan
loan_manager = DataLoader()
loan_manager.load_feather(loan_feather_path)

/home/mattkhoo/Git-Loan-Repayment-EDA-Predictive/data-feathers/loan


In [24]:
# Display df info (comment accordingly)
loan_manager.display(10, True)  # first n rows
loan_manager.display(10, False) # last n rows

,loanId,anon_ssn,payFrequency,apr,applicationDate,originated,originatedDate,nPaidOff,approved,isFunded,loanStatus,loanAmount,originallyScheduledPaymentAmount,state,leadType,leadCost,fpStatus,clarityFraudId,hasCF
0,LL-I-07399092,beff4989be82aab4a5b47679216942fd,B,360.00,2016-02-23T17:29:01.940000,False,None,0.0,False,0,Withdrawn Application,500.0,978.27,IL,bvMandatory,6,None,5669ef78e4b0c9d3936440e6,1
1,LL-I-06644937,464f5d9ae4fa09ece4048d949191865c,B,199.00,2016-01-19T22:07:36.778000,True,2016-01-20T15:49:18.846000,0.0,True,1,Paid Off Loan,3000.0,6395.19,CA,prescreen,0,Checked,569eb3a3e4b096699f685d64,1
2,LL-I-10707532,3c174ae9e2505a5f9ddbff9843281845,B,590.00,2016-08-01T13:51:14.709000,False,None,0.0,False,0,Withdrawn Application,400.0,1199.45,MO,bvMandatory,3,None,579eab11e4b0d0502870ef2f,1
3,LL-I-02272596,9be6f443bb97db7e95fa0c281d34da91,B,360.00,2015-08-06T23:58:08.880000,False,None,0.0,False,0,Withdrawn Application,500.0,1074.05,IL,bvMandatory,3,None,555b1e95e4b0f6f11b267c18,1
4,LL-I-09542882,63b5494f60b5c19c827c7b068443752c,B,590.00,2016-06-05T22:31:34.304000,False,None,0.0,False,0,Rejected,350.0,814.37,NV,bvMandatory,3,None,5754a91be4b0c6a2bf424772,1
5,LL-I-09734486,b5541f49472fa0fce8e473306768f7fb,M,650.00,2016-06-12T19:27:47.951000,False,None,0.0,False,0,Withdrawn Application,300.0,738.18,IN,organic,0,None,574e2029e4b061d2c3a16a4c,1
6,LL-I-15006968,1828c64bb2dffeae88b27174a9f79ecc,B,680.00,2017-01-12T18:04:37.921000,False,None,0.0,False,0,Withdrawn Application,400.0,1362.92,TX,lead,25,None,5877c533e4b08f2480ab7e5a,1
7,LL-I-08327946,02596517e7633c7e87e6b333a0fb1bbe,M,449.99,2016-04-01T22:13:02.131000,False,None,0.0,False,0,Withdrawn Application,350.0,759.84,UT,bvMandatory,3,None,None,0
8,LL-I-10568307,523ed92e1145eb688bb631da24695197,S,251.00,2016-07-27T00:19:52.808000,False,None,0.0,False,0,Withdrawn Application,2600.0,8230.01,CA,california,40,None,5797fe22e4b0d05020f298ce,1
9,LL-I-05518977,47bf79119075e41ef65510f2900c8e4a,B,360.00,2015-12-09T18:17:33.622000,False,None,0.0,False,0,Withdrawn Application,500.0,995.22,IL,bvMandatory,6,None,563281d2e4b07887adc5bc0a,1
